In [60]:
import pandas as pd
data_path = '../raw_data/'
df = pd.read_excel(data_path + 'HCP Dataset for Case Study.xlsx')


# Data cleaning with feature engineering

In [61]:
# Get dates and time into a workable datetime format
df['AdmissionDateTime'] = pd.to_datetime(df['AdmissionDate'].astype(str).str.zfill(8), format='%d%m%Y') + pd.to_timedelta(df['AdmissionTime'].astype(str).str.zfill(4).str[:2] + ':' + df['AdmissionTime'].astype(str).str.zfill(4).str[2:] + ':00')
df['SeparationDateTime'] = pd.to_datetime(df['SeparationDate'].astype(str).str.zfill(8), format='%d%m%Y') + pd.to_timedelta(df['SeparationTime'].astype(str).str.zfill(4).str[:2] + ':' + df['AdmissionTime'].astype(str).str.zfill(4).str[2:] + ':00')
df['DateOfBirth'] = pd.to_datetime(df['DateOfBirth'].astype(str).str.zfill(8), format='%d%m%Y')


In [62]:
# Calculate time in hours for each episode (from admission to separation)
df['EpisodeDurationHours'] = (df['SeparationDateTime'] - df['AdmissionDateTime']).dt.total_seconds() / 3600

In [63]:
# transform date of birth into age as of admission date
df['Age'] = (df['AdmissionDateTime'] - df['DateOfBirth']).dt.days // 365

In [64]:
len(df[df['PrincipalDiagnosis'].astype(str).str.len() != 6]['PrincipalDiagnosis'].unique())

11

In [65]:
# Principal Diagnosis Code Anomolies
# get unique codes where length is not 6 (according to Data Specs)
print(df[df['PrincipalDiagnosis'].astype(str).str.len() != 6]['PrincipalDiagnosis'].value_counts().reset_index())
unique_anomoly_codes = df[df['PrincipalDiagnosis'].astype(str).str.len() != 6]['PrincipalDiagnosis'].unique()
# replace the anomolous codes with separate dummy codes for ease of use
replace_map = {
    unique_anomoly_codes[i] : f'0A00{str(i).zfill(2)}' for i in range(len(unique_anomoly_codes))
}
df['PrincipalDiagnosis'] = df['PrincipalDiagnosis'].replace(replace_map)


                                   PrincipalDiagnosis  count
0   1999999999999999854227564838689211149197336306...     16
1         2000000000000000089771425356151833571098624     11
2   1999999999999999984180772525672677016455122433...      9
3          200000000000000001240017290081556638990336      3
4   1999999999999999888335104945098667625499457407...      2
5   1999999999999999968690075053594844794467049550...      2
6   1999999999999999986441897348723255952923416883...      2
7   2000000000000000029261390461349746061940085975...      1
8                   200000000000000010732324408786944      1
9   2000000000000000115715919885453939654786757378...      1
10  2000000000000000065564491965724196497141410566...      1


In [66]:
print(df[df['PrincipalDiagnosis'].astype(str).str.len() != 6]['PrincipalDiagnosis'].value_counts().reset_index())


Empty DataFrame
Columns: [PrincipalDiagnosis, count]
Index: []


In [71]:
sex_map = {
    1 : 'Male',
    2 : 'Female',
    3 : 'Other',
    9 : 'N/A'
}
df['Sex'] = df['Sex'].replace(sex_map)

In [76]:
# include only columns of interest for ease of use
columns_of_interest = [
    'EpisodeIdentifier',
    'InsurerIdentifier',
    'Age',
    'Sex',
    'AdmissionDateTime',
    'SeparationDateTime',
    'EpisodeDurationHours',
    'PrincipalDiagnosis',
]
df_new = df[columns_of_interest]

In [77]:
df_new

,EpisodeIdentifier,InsurerIdentifier,Age,Sex,AdmissionDateTime,SeparationDateTime,EpisodeDurationHours,PrincipalDiagnosis
0,1624122,INS1,23,Male,2023-01-01 08:50:00,2023-01-02 09:50:00,25.0,2S620
1,1624177,INS5,86,Female,2023-01-01 13:30:00,2023-01-06 10:30:00,117.0,2J22
2,1624113,INS9,55,Female,2023-01-01 11:55:00,2023-01-09 11:55:00,192.0,2S8282
3,1624005,,68,Male,2023-02-01 10:37:00,2023-02-01 14:37:00,4.0,2Z511
4,1625135,,60,Male,2023-02-01 13:07:00,2023-02-01 14:07:00,1.0,2Z511
...,...,...,...,...,...,...,...,...
30610,1624133,INS1,72,Male,2022-12-31 15:00:00,2023-01-05 11:00:00,116.0,2I489
30611,1624139,INS1,94,Female,2022-12-31 16:30:00,2023-02-01 13:30:00,765.0,2F058
30612,1622697,INS2,55,Male,2022-12-31 13:25:00,2022-12-31 19:25:00,6.0,2Z491
30613,1622749,INS5,80,Male,2022-12-31 11:40:00,2022-12-31 16:40:00,5.0,2Z491
